In [1]:
# Import BS4
from bs4 import BeautifulSoup
import requests
import re
from IPython.display import display, HTML

def get_soup(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError('Failed to fetch web page with status code: {}'.format(response.status_code))
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup



In [5]:
url = "https://www.linkedin.com/jobs/view/4023953957/?eBP=CwEAAAGSF707ObQCMY82VUMK3ANR3h_axbkGZjGbdhm7XAXPqohuwQsLyfHMwRmrGRNolcKSmAsFGjHSzQsLhp8wcKwqpYR8ok1ymzgYS9xrVQT3njRv2ClHmFQ_92EbPOZBxGraUzrf6-FkvN581EDJLnn6tZmINeNTYn72x3fo1PJCDQcWA1dT2dydE9PzM19Waa9wxvnRThQMOfRsc4kNNOZEPi8W_9wE0A7z8YXE7WF7_8jCa-OkNPUmuPHamOA9XhwP_HS5IzqxE2zUJDHNLhn2F2JqPbxXqohFV2OnQfBdiwc7J3lmCXpMbyRdCc9YxW0e4PaMs9EYXsj7zcHN5ZZ0TatQ0dOOgqVJAU3vfk5sB3IbJZxXLoPOERrsnzi74F8mgdGV0qHjJuhn_nT7VAZU-agf850Jy5HwXNt8OaksRAcs4TunapyfDJaRv"
# url = "https://www.linkedin.com/jobs/view/4029494858/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=5eWuEhVJjv1DTtpUZ%2B6zYA%3D%3D&trackingId=eDKfaKdvhTSNvWRbOJxCfg%3D%3D&trk=flagship3_search_srp_jobs"
soup = get_soup(url)
HTML(soup.prettify())


In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="d_jobs_guest_details" name="pageKey"/>
  <!-- -->
  <!-- -->
  <meta content="en_US" name="locale"/>
  <meta data-app-version="2.0.1974" data-browser-id="39bfd7ed-44c7-43f3-83ac-d528d10d5dcb" data-call-tree-id="AAYirqChGJQ1vdRA9YMncg==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="control" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_details;jAa0uE2/TPGFDiOC3Y9TbA==" data-service-name="jobs-guest-frontend" id="config"/>
  <link href="https://ca.linkedin.com/jobs/view/software-developer-co-op-at-varian-4029494858" rel="canonical"/>
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
  <script>
   function get

In [3]:
def parse_job_posting(soup):
    company_selector = "div.sub-nav-cta__sub-text-container > a"
    loaction_selector = "div.sub-nav-cta__sub-text-container > span"
    job_title_selector = "div.sub-nav-cta__text-container > h3.sub-nav-cta__header"
    job_description_selector = "div.description__text > section > div"

    company = soup.select(company_selector)[0].text.strip()
    location = soup.select(loaction_selector)[0].text.strip()
    job_title = soup.select(job_title_selector)[0].text.strip()
    job_description = "\n".join([child.text for child in soup.select(job_description_selector)[0].children])
    job_description = re.sub(r"\n+", "\n", job_description).strip()

    job_posting_data = {
        "job_title": job_title,
        "company": company,
        "location": location,
        "job_description": job_description
    }
    return job_posting_data

job_posting_data = parse_job_posting(soup)
job_posting_data

{'job_title': 'Software Developer [Co-op]',
 'company': 'Varian',
 'location': 'Winnipeg, Manitoba, Canada',
 'job_description': "Together, we can beat cancer.\nAt Varian, a Siemens Healthineers Company, we bring together the world's best talent to realize our vision of a world without fear of cancer. Together, we work passionately to develop and deliver easy-to-use, efficient oncology solutions.\nWe are part of an incredible community of scientists, clinicians, developers, researchers, professionals, and skilled specialists pushing the boundaries of what’s possible, to improve people’s lives around the world. We embrace a culture of inclusivity in which the power and potential of every individual can be unleashed. We spark ideas that lead to positive impact and continued success.\nIf you want to be part of this important mission, we want to hear from you.\nIn your role as Cloud Software Developer at Varian, a Siemens Healthineers Company, you are responsible for the development and ma

In [ ]:
def import_from_url (url):
    try:
        soup = get_soup(url)
        job_posting_data = parse_job_posting(soup)
        return job_posting_data
    except Exception as e:
        return f"Failed to import from url due to {str(e)}"